In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_auc_score

# Obtained after training
model = load_model('deepfake_cnn_model.h5')

def preprocessImage(imgPath):
    img = image.load_img(imgPath)  # Assuming preprocessing of 256x256
    imgArray = image.img_to_array(img)  # Convert to numpy array
    imgArray = imgArray / 255.0        # Normalize (same as training)
    imgArray = np.expand_dims(imgArray, axis=0)  # Add batch dimension
    return imgArray


In [31]:
# SINGLE IMAGE TEST

# INPUT IMAGE PATH HERE
imgPath = r"" 
inputImage = preprocessImage(imgPath)

prediction = model.predict(inputImage)[0][0]

# Interpret result
if prediction < 0.5:
    print(f"AI-generated (Deepfake) — Confidence: {1 - prediction:.4f}")
else:
    print(f"Real Image — Confidence: {prediction:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
AI-generated (Deepfake) — Confidence: 0.9574


In [ ]:
# DATASET REDUCTION

data_dir = "./Dataset/Test"
all_files = []
for cls in ['Real', 'Fake']:
    class_path = os.path.join(data_dir, cls)
    files = [os.path.join(cls, f) for f in os.listdir(class_path) if f.endswith('.jpg')]
    all_files.extend(files)

# Shuffle and take 10%
np.random.seed(123)
np.random.shuffle(all_files)
test_files = all_files[:int(0.1 * len(all_files))]

In [ ]:
# CALCULATING AREA UNDER CURVE

testDatagen = ImageDataGenerator(rescale=1./255)

testGenerator = testDatagen.flow_from_directory(
    './Dataset/Test',
    target_size=(256, 256),  # match your model's expected input size
    batch_size=32,
    class_mode='binary',
    shuffle=False  # important for matching predictions with labels
)

yPredProbs = model.predict(testGenerator)
yTrue = testGenerator.classes

auc = roc_auc_score(yTrue, yPredProbs)
print(f"AUC: {auc:.4f}")